In [1]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/umasreeram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/umasreeram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
pip install langdetect

Note: you may need to restart the kernel to use updated packages.


__Creating functions to clean tweets__

In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer
#import lang_detect
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from langdetect import detect

def stem_and_lemmatize(words):
    stems = stem_words(words)
    return " ".join(stems)


def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = LancasterStemmer()
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def lemmatize_verbs(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmatizer = WordNetLemmatizer()
    lemmas = []
    for word in words:
        lemma = lemmatizer.lemmatize(word, pos='v')
        lemmas.append(lemma)
    return lemmas

# def stem_and_lemmatize(words):
#     stems = stem_words(words)
#     return " ".join(stems)
#     lemmas = lemmatize_verbs(stems)
#     return " ".join(lemmas)

def only_english(mystr):
    if lang_detect(mystr)=='en':   #give up
        return mystr
    else :
        return ''


def clean_string(mystr):
    mystr=mystr.lower()
    mystr=re.sub(r"\\\w+", " ", mystr)
    mystr=re.sub(r"\@\w+"," ",mystr)
    mystr=re.sub(r"\#\w+"," ",mystr)
    mystr=re.sub(r"https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)"," ",mystr)
    
    mystr=mystr[2:-1]
    
    cleantext = "".join([x.lower() if (x.isalpha() or x.isspace()) else ' ' for x in mystr])
    return cleantext

def remove_stop_words(mystr):
    word_list= mystr.split()
    cleaned_word_list=[w for w in word_list if w not in stop_words and len(w)>=2]
    
    return cleaned_word_list
   
    
def join_sep(mylist):
    return " ".join(mylist)
   


In [73]:
#Important columns stored after feature selection in impcolumns.txt

with open('impcolumns.txt', 'r') as f:
    impcolumns = f.readlines()
    

    
impcolumns = [line[:-1] for line in impcolumns]
#print(impcolumns)
vertical_stack=impcolumns

__Loading data and packages__

In [74]:
#f= approach_FORGRAPH_tweets_data.json'
import json
with open('approach4_final_data.json', 'r') as fp:
    tweet_data0 = json.load(fp)
    
    
with open('approach2_FORGRAPH1.json', 'r') as fp:
    tweet_data1 = json.load(fp)
    
with open('approach2_FORGRAPH2.json', 'r') as fp:
    tweet_data2 = json.load(fp)

In [43]:
#Using oversampled data from drive

In [44]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

__Loading Classifier and prediction__

In [45]:
import pickle
from sklearn.externals import joblib

In [46]:
Pkl_Filename='Classifiers/DT_adaboost.pkl'
with open(Pkl_Filename, 'rb') as file:  
    classifier = clf = joblib.load(file)


classifier

/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=None,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                       

__Extras__

In [47]:
# X_new.to_csv('X_train_oversampled.csv', index = False, header=True)
# y_new.to_csv('y_train_oversampled.csv', index = False, header=True)
# X_test.to_csv('X_test.csv', index = False, header=True)
# y_test.to_csv('y_test.csv', index = False, header=True)

### Making tfidf from tweets and score


In [78]:


def return_score(user_tweets):     #here user_tweets would be a dict with key as the username and value as the tweets 

     
    dataframes = {}
    dataframes_empty = {}


    for x in user_tweets:
    #print(x)
        
    
        try:
            file=pd.DataFrame(user_tweets[x],columns=['id','text'])
    
            file['clean_text']=file['text'].apply(clean_string)
            file['word_list']=file['clean_text'].apply(remove_stop_words)
            file['word_list']=file['word_list'].apply(stem_and_lemmatize)
            #file['word_list']=file['word_list'].apply(join_sep)
            #file['word_list']=file['word_list'].apply(only_english)
    
            file=file.dropna()
            file=file.loc[file['word_list']!='']
            
            ##############
    
    
            vectorizer = TfidfVectorizer()
        
            tfidfmatrix = vectorizer.fit_transform(file['word_list'])
            vocab = vectorizer.get_feature_names()

            tfidf_data=tfidfmatrix.toarray()
        
            tfidf_pd=pd.DataFrame(data=tfidf_data,columns=vocab,index=file['id'])
        
            tfidf_pd = tfidf_pd.drop(tfidf_pd.columns.difference(impcolumns),1)
       
        
            dataframes[x] = tfidf_pd
            #print("Success",x)
    
        except ValueError as e: #happens when the vocabulary is empty
        
            #print("Failure",x,file['text'])
            dataframes_empty[x]='N/A'
        
        
            continue
            
    tesla_followers=dataframes
    feature_list=impcolumns
    #UNIFORM PADDING
    for key in tesla_followers:
        test=tesla_followers[key]
    
        #print(len(test.columns))
    

        toadd=list(set(feature_list)-set(test.columns))


        toadd_df=pd.DataFrame(np.zeros((test.shape[0], len(toadd))),columns=toadd)
        test.reset_index(drop=True, inplace=True)
        toadd_df.reset_index(drop=True, inplace=True)


        test=pd.concat([test,toadd_df],axis=1)
    
        tesla_followers[key]=test
        #print(test.shape)

    
    #SCORE    
    tesla_tweet_prediction=[]
    score={}
    for key in tesla_followers:
        test=tesla_followers[key]
        current_user_pred=classifier.predict(test)
        tesla_tweet_prediction.append(current_user_pred)
        score[key]=sum(current_user_pred)/len(current_user_pred)   
    

    import collections
    from collections import OrderedDict
    from operator import itemgetter 
    sorted_score = OrderedDict(sorted(score.items(), key=itemgetter(1),reverse=True))
    sorted_score.update(dataframes_empty)    #adding users with No tweets
    
    return sorted_score




In [80]:
first_level={}
first_no_of_tweets={}
f_list=[]

ctr=0


for k, v in tweet_data0['tesla'].items():
    #print(k)
    #print(v.keys())
    if k not in first_level:
        first_level[k]=v['tweets']
        f_list.append(k)
        first_no_of_tweets[k]=len(v['tweets'])
    
for k, v in tweet_data1['tesla'].items():
    #print(k)
    #print(v.keys())
    if k not in first_level:
        first_level[k]=v['tweets']
        f_list.append(k)
        first_no_of_tweets[k]=len(v['tweets'])
    
for k, v in tweet_data2['tesla'].items():
    #print(k)
    #print(v.keys())
    if k not in first_level:
        first_level[k]=v['tweets']
        f_list.append(k)
        first_no_of_tweets[k]=len(v['tweets'])
    

    
print(len(first_no_of_tweets))
#print(f_list)

405


In [91]:
first_level={}

for k, v in tweet_data['tesla'].items():
    #print(k)
    #print(v.keys())
    first_level[k]=v['tweets']

In [92]:

first_level_list= return_score(first_level)
    

In [83]:
second_level={}
for k, v in tweet_data['tesla'].items():
    second_level[k] = {}
    follwer_list = []
    user_tweets={}
    if isinstance(v, dict):
        for i, j in v.items():
            
            if not (i == "tweets" or i == 'location'):
                    follwer_list.append(i)
                    if j['tweets']!="error":
                        user_tweets[i] = j['tweets']
    second_level[k] = return_score(user_tweets)
                        

                            

In [84]:
g=0
ctr=0
for x in second_level:
    
    #print(len(second_level[x]))
    g+=len(second_level[x])
    
    
print(g)

2903


In [85]:

import json
with open('hierarchy_scores.json', 'w') as fp:
    json.dump(second_level, fp)

In [86]:
#KMEANS
from sklearn.cluster import KMeans

def kmeans_avg(scores_list):
    X = np.array(scores_list)
    if len(scores_list)>5:
        kmeans = KMeans(n_clusters=3, random_state=0).fit(X.reshape(-1, 1))
        label = kmeans.labels_
        counts = np.bincount(label)
        cluster_no = (np.argmax(counts))
        result = np.array(list(zip(X,label)))
        res = result[result[:,1] == cluster_no]
        web_score = np.mean(res[:,0])
    else:
        #print(scores_list)
        web_score=np.mean(scores_list)
    
    return web_score



In [87]:
second_level_list={}

for x,y in second_level.items():
    try:
        if not(y.values()==[] or y.values()==['N/A']):
            second_level_list[x]=kmeans_avg(list(y.values()))
        else:
            second_level_list[x]='N/A(web)'
    except:
        second_level_list[x]='N/A(web)'
    
    
    
print(second_level_list)

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean o

{'kenziemelonn': 0.3519369246262211, 'AguonJojo': nan, 'orkunalbayrakk': nan, 'chuanlongwu': nan, 'stahl_waylon': nan, 'LightTheoryLLC': 'N/A(web)', 'TonyLGM': 0.32463090612219025, 'MdSiddi2': nan, 'TheBuffBengali': 0.26877030630922916, 'kutbuImus': 'N/A(web)', 'rongron22700921': 0.42908697221219777, 'akash2396': nan, 'Surajit31177051': 0.41145833333333337, 'SkiljoDenis': nan, 'JeevaVj96231590': 0.23537581699346405, 'MarkinhoMaciel': 'N/A(web)', 'clot503': nan, 'Yhimna1': nan, 'cObW3Etwih6JjOl': 0.507433489827856, 'StartAHealtyLif': 0.5403783440418197, 'RitejGaba': 0.5113271019912662, 'Netassa2': 0.3787878787878788, 'satyamjigurjar': nan, 'AmirRakim': nan, 'willthompson94': 0.31080920845651383, 'JoshuaWinkelma5': 0.4665158371040724, 'AfxcSack': 0.298196574100233, 'jason71805778': nan, 'geewu': 'N/A(web)', 'SrikarKolli': 0.43915343915343913, 'ShawnStack1': nan, '_Arnab_': 0.33250412510099053, 'yophooey': 0.35255824577432615, '14_rjay': 'N/A(web)', 'nadhirahbaharin': 0.3623077765190306, 

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [93]:
comparison={}

for x in first_level:
    comparison[x]=list([first_no_of_tweets[x],first_level_list[x],second_level_list[x]])

In [94]:
#print(comparison)

threshold=0.5

comp_pd=pd.DataFrame.from_dict(data=comparison)
comp_pd=comp_pd.transpose()
comp_pd.columns=['#Tweets','Text based','Web based'] 
comp_pd = comp_pd[comp_pd['Text based'].apply(lambda x: isinstance(x, float) )]
comp_pd = comp_pd[comp_pd['Web based'].apply(lambda x: isinstance(x, float)   )]
comp_pd = comp_pd.dropna()

comp_pd.loc[(comp_pd['Text based'] < threshold),'T Result']='No'
comp_pd.loc[(comp_pd['Text based'] >= threshold),'T Result']='Yes'
comp_pd.loc[(comp_pd['Web based'] < threshold),'W Result']='No'
comp_pd.loc[(comp_pd['Web based'] >= threshold),'W Result']='Yes'

comp_pd.loc[(comp_pd['Text based'] < threshold) & (comp_pd['Web based'] < threshold) , 'Match'] = 'Yes'
comp_pd.loc[(comp_pd['Text based'] >= threshold) & (comp_pd['Web based'] >= threshold) , 'Match'] = 'Yes'
comp_pd.loc[(comp_pd['Text based'] >= threshold) & (comp_pd['Web based'] < threshold) , 'Match'] = 'No'
comp_pd.loc[(comp_pd['Text based'] < threshold) & (comp_pd['Web based'] >= threshold) , 'Match'] = 'No'

comp_pd['Text based']=comp_pd['Text based'].apply(lambda x: round(x,3))
comp_pd['Web based']=comp_pd['Web based'].apply(lambda x: round(x,3))

#print(comp_pd)
comp_pd.to_csv('comparison.csv',encoding='utf-8')

from sklearn.metrics import confusion_matrix
y_true=list(comp_pd['T Result'])
y_pred=list(comp_pd['W Result'])

print(confusion_matrix(y_true,y_pred).ravel())  # Confusion matrix printed






[42  5 20  5]
